In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib as mpl

from common_functions import foo


mpl.style.use('classic')

%load_ext autoreload
%autoreload 2

results = "results"

In [2]:
lastKDays = 30
df = foo(lastKDays, results, True)
df.groupby(["reg"]).mean()

,kout,kin,day,rmse,lastKDays,seed
reg,,,,,,
gp,2.498016,2.001424,163.983420,20.665598,30.0,-1.000000
gp_rd,2.499959,1.999573,163.968984,24.380714,30.0,2.000305
knn,2.500000,2.000000,164.000000,30.679426,30.0,-1.000000
knn_rd,2.500000,2.000000,164.000000,30.541583,30.0,2.000000
qbc,2.500000,2.000000,164.000000,22.671395,30.0,-1.000000
qbc_rd,2.500000,2.000000,164.000000,24.136366,30.0,2.000000


In [3]:
krig_results = pd.read_csv("active_kriging_results.csv")
krig_results.head()

,kout,kin,rmse
0,0,0,15.107037
1,0,1,15.960798
2,0,2,16.528570
3,0,3,15.294778
4,0,4,16.662367


In [4]:
splits = 6
tdic = {}
for reg in ['gp', 'gp_rd', 'qbc', 'knn', 'knn_rd', 'qbc_rd']:
    for lastKDays in [30]:
        df = foo(lastKDays, results, True)
        days = df['day'].unique()
        for ix, tempdf in df.groupby(["kout", "kin"]):
            kout, kin = ix
            a = krig_results[(krig_results["kout"] == kout)  & (krig_results["kin"] == kin)]["rmse"].values
            if sum(np.isnan(a)) != 0:
                raise ValueError
            b =  tempdf[tempdf["reg"] == reg]["rmse"].values
            if sum(np.isnan(b)) != 0:
                raise ValueError
            a = a.mean()
            b = b.mean()
            if a <= b:
                temp = tdic.get(reg, [0, 0])
                tdic[reg] = [temp[0] + 1, temp[1]]
            else:
                temp = tdic.get(reg, [0, 0])
                tdic[reg] = [temp[0], temp[1] + 1]

    print (f"{reg} Done.")

gp Done.
gp_rd Done.
qbc Done.
knn Done.
knn_rd Done.
qbc_rd Done.


In [5]:
tdic

{'gp': [29, 1],
 'gp_rd': [30, 0],
 'qbc': [30, 0],
 'knn': [30, 0],
 'knn_rd': [30, 0],
 'qbc_rd': [30, 0]}